# Tiền xử lí dữ liệu

In [1]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sb

In [2]:
df = pd.read_csv('data.csv')
df.head()

,cum,sbd,toan,ngu_van,ngoai_ngu,vat_li,hoa_hoc,sinh_hoc,lich_su,dia_ly,gdcd
0,1,1000001,3.6,5.00,4.0,NaN,NaN,NaN,2.75,6.0,8.75
1,1,1000002,8.4,6.75,7.6,NaN,NaN,NaN,8.50,7.5,8.25
2,1,1000003,5.8,7.50,5.0,NaN,NaN,NaN,7.25,5.5,8.75
3,1,1000004,7.4,7.50,8.6,NaN,NaN,NaN,7.50,6.5,7.50
4,1,1000005,7.2,8.50,9.0,NaN,NaN,NaN,8.00,8.5,8.25


### **Nhận xét:**
Cơ bản sẽ có 4 đối tượng thi:
- **Khóa 2022 thi THPTQG:**

    - Khối khoa học tự nhiến: Toán Văn Anh Lý Hóa Sinh --- 6 môn
    - Khôi khoc học xã hội  : Toán Văn Anh Sử Địa GDCD --- 6 môn  
    
- **Giáo dục thường xuyên xét tốt nghiệp + đại học:**

    - Thi 3 môn tổ hợp khtn hoặc khxh
    - Không thi tiếng anh
    - Tổng thi: 5 môn
    - VD: KHTN: Toán Văn  Lý Hóa Sinh
    - VD: KHXH: Toán Văn  Sử Địa GDCD

- **Khóa 2021 về trước thi lại đại học (Đã tốt nghiệp THPT)**

    - Thi một khối duy nhất (Vd: khối A - Toán Lý Hóa)         --- 3  môn
    - Thi nhiều hơn một khối (Vd: khối A B - Toán Lý Hóa Sinh) --- 3 < số môn < 6

- **Khóa 2021 về trước thi lại tốt nghiệp (Chưa tốt nghiệp):**

    - Một số môn năm trước đủ cao và đã bảo lưu
    - Chỉ thi những môn còn thiếu để xét tốt nghiệp
    - Nếu thi >= 3 môn thì có thể lấy điểm 3 môn đó xét điểm ĐH nên đưa vào trường hợp phía trên
    - Chỉ thi 1 - 2 môn
    

### **1. Kiểm tra kiểu dữ liệu**

In [3]:
print(df.dtypes)

cum            int64
sbd            int64
toan         float64
ngu_van      float64
ngoai_ngu    float64
vat_li       float64
hoa_hoc      float64
sinh_hoc     float64
lich_su      float64
dia_ly       float64
gdcd         float64
dtype: object


### **2. Xác định các trường hợp xảy ra**

In [4]:
df.count()

cum          995441
sbd          995441
toan         982726
ngu_van      981407
ngoai_ngu    870609
vat_li       325523
hoa_hoc      327367
sinh_hoc     322198
lich_su      659662
dia_ly       657421
gdcd         554343
dtype: int64

In [26]:
isNan_score,  isNan_counts = np.unique(np.isnan(df.loc[:, 'toan':]), return_counts=True, axis=0)
print(isNan_score)
print(len(isNan_counts))
print(isNan_counts)

[[False False False False False False  True  True  True]
 [False False False False False  True  True  True  True]
 [False False False False  True False  True  True  True]
 [False False False False  True  True  True  True  True]
 [False False False  True False False  True  True  True]
 [False False False  True False  True  True  True  True]
 [False False False  True  True False  True  True  True]
 [False False False  True  True  True False False False]
 [False False False  True  True  True False False  True]
 [False False False  True  True  True False  True False]
 [False False False  True  True  True False  True  True]
 [False False False  True  True  True  True False False]
 [False False False  True  True  True  True False  True]
 [False False False  True  True  True  True  True False]
 [False False False  True  True  True  True  True  True]
 [False False  True False False False  True  True  True]
 [False False  True False False  True  True  True  True]
 [False False  True False  True

- *Có thể thấy số lượng thí sinh của từng môn đề khác nhau. Điều này xảy ra là do những thí sinh thi tự do và chỉ thi những môn mà mình mong muốn.*
- *Số trường hợp xảy ra lên đến 101 TH, bây giờ em sẽ đi sắp xếp các trường hợp đó theo 4 đối tượng đã nêu ở phía trên*

##### ***2.1 Số lượng thí sinh thi 7/8/9 môn (không thi 2/1/0 môn)***

In [14]:
print(isNan_score[np.sum(isNan_score, axis=1) == 0])
print(isNan_score[np.sum(isNan_score, axis=1) == 1])
print(isNan_score[np.sum(isNan_score, axis=1) == 2])

[]
[]
[]


- Tất cả đều trả về 0 thí sinh.
- Điều này cũng dễ hiểu vì tổ hợp tự nhiên (lí, hóa, sinh) và xã hội (sử, địa, gdcd) được tổ chức cùng ngày, cùng giờ. Nên không thể có thí sinh nào thi trên 6 môn.

##### ***2.2 Số lượng thí sinh thi 6 môn (không thi 3 môn)***

In [22]:
print(isNan_score[np.sum(isNan_score, axis=1) == 3])
print(np.sum(isNan_counts[np.sum(isNan_score, axis=1) == 3]))

[[False False False False False False  True  True  True]
 [False False False  True  True  True False False False]]
858736


- Có 858,736 thí sinh.
- Đây chính là tổng số tất cả thí sinh thi THPTQG2022
- Bắt buộc thi 2 khối: KHTN và KHXH
    - Toán Văn Anh Lý Hóa Sinh
    - Toán Văn Anh Sử Địa GDCD

##### ***2.2 Số lượng thí sinh thi 5 môn (không thi 4 môn)***

In [35]:
Giao_duc_thuong_xuyen = [[False, False, True, False, False, False, True, True, True],\
                         [False, False, True, True, True, True, False, False, False]]
print(isNan_score[np.sum(isNan_score, axis=1) == 4])
print(np.sum(isNan_counts[np.sum(isNan_score, axis=1) == 4]))
print(np.sum(isNan_counts[np.sum(isNan_score == Giao_duc_thuong_xuyen[0], axis=1)])) # Error


[[False False False False False  True  True  True  True]
 [False False False False  True False  True  True  True]
 [False False False  True False False  True  True  True]
 [False False False  True  True  True False False  True]
 [False False False  True  True  True False  True False]
 [False False False  True  True  True  True False False]
 [False False  True False False False  True  True  True]
 [False False  True  True  True  True False False False]
 [False  True False False False False  True  True  True]
 [False  True False  True  True  True False False False]
 [ True False False  True  True  True False False False]]
12756
6920721


### **3. Tạo cột mới: khoi ('khtn' or 'khxh')**

In [8]:
science = 'science'
social = 'social'
retest = 'retest'
# point vat_li >= 0  or hoa_hoc >= 0 or sinh_hoc >= 0 => science
# point dia_li >= 0  or su >= 0 or dia_lich >= 0 = > social
# Else => NaN

df['khoi'] = df.apply(lambda row: science if (row['vat_li'] >= 0 or row['hoa_hoc'] >= 0 or row['sinh_hoc'] >= 0) \
                            else social if (row['dia_ly'] >= 0 or row['lich_su'] >= 0 or row['gdcd'] >= 0) \
                            else retest, axis=1)

In [9]:
df[0:100]

,cum,sbd,toan,ngu_van,ngoai_ngu,vat_li,hoa_hoc,sinh_hoc,lich_su,dia_ly,gdcd,khoi
0,1,1000001,3.6,5.00,4.0,NaN,NaN,NaN,2.75,6.00,8.75,social
1,1,1000002,8.4,6.75,7.6,NaN,NaN,NaN,8.50,7.50,8.25,social
2,1,1000003,5.8,7.50,5.0,NaN,NaN,NaN,7.25,5.50,8.75,social
3,1,1000004,7.4,7.50,8.6,NaN,NaN,NaN,7.50,6.50,7.50,social
4,1,1000005,7.2,8.50,9.0,NaN,NaN,NaN,8.00,8.50,8.25,social
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,1000099,7.6,8.00,9.2,NaN,NaN,NaN,5.00,8.50,8.00,social
96,1,1000100,3.8,5.50,5.6,NaN,NaN,NaN,5.50,4.25,7.75,social
97,1,1000101,8.2,6.50,7.6,5.25,7.25,3.25,NaN,NaN,NaN,science
98,1,1000102,8.8,7.25,9.0,4.75,8.00,8.00,NaN,NaN,NaN,science


### **4. Tách dữ liệu thành 2 phần: KHTN và KHXH**

In [16]:
# KHTN
pf_khtn = df[df['khoi'] == science]
pf_khtn = pf_khtn.drop(['lich_su', 'gdcd', 'dia_ly'], axis=1)
# pf_khtn = pf_khtn.fillna(0)


# KHXH
pf_khxh = df[df['khoi'] == social]
pf_khxh = pf_khxh.drop(['vat_li', 'hoa_hoc', 'sinh_hoc'], axis=1)

# Thi lai
pf_retest = df[df['khoi'] == retest]

In [17]:
pf_khtn.head()
# count row
pf_khtn.count()

cum          329235
sbd          329235
toan         329205
ngu_van      315709
ngoai_ngu    312709
vat_li       325523
hoa_hoc      327367
sinh_hoc     322198
khoi         329235
dtype: int64

In [18]:
pf_khxh.head()

,cum,sbd,toan,ngu_van,ngoai_ngu,lich_su,dia_ly,gdcd,khoi
0,1,1000001,3.6,5.00,4.0,2.75,6.0,8.75,social
1,1,1000002,8.4,6.75,7.6,8.50,7.5,8.25,social
2,1,1000003,5.8,7.50,5.0,7.25,5.5,8.75,social
3,1,1000004,7.4,7.50,8.6,7.50,6.5,7.50,social
4,1,1000005,7.2,8.50,9.0,8.00,8.5,8.25,social


In [22]:
pf_retest.head()

,cum,sbd,toan,ngu_van,ngoai_ngu,vat_li,hoa_hoc,sinh_hoc,lich_su,dia_ly,gdcd,khoi
59,1,1000060,8.2,7.50,7.2,NaN,NaN,NaN,NaN,NaN,NaN,retest
69,1,1000070,6.4,6.50,7.6,NaN,NaN,NaN,NaN,NaN,NaN,retest
145,1,1000149,5.0,8.25,8.0,NaN,NaN,NaN,NaN,NaN,NaN,retest
166,1,1000170,7.4,7.00,8.0,NaN,NaN,NaN,NaN,NaN,NaN,retest
201,1,1000205,8.2,9.00,8.8,NaN,NaN,NaN,NaN,NaN,NaN,retest


In [21]:
for sv in pf_retest['sbd']:
    print(sv)

1000060
1000070
1000149
1000170
1000205
1000321
1000372
1000382
1000386
1000402
1000422
1000459
1000471
1000476
1000487
1000490
1000519
1000522
1000525
1000548
1000550
1000554
1000555
1000923
1000950
1000983
1000998
1001055
1001076
1001112
1001118
1001458
1001539
1003438
1003524
1003559
1003606
1003657
1003663
1003684
1003689
1003717
1003718
1003768
1003775
1003778
1003786
1003798
1003799
1003803
1003846
1004936
1005874
1006001
1006031
1006062
1006067
1006074
1006075
1006133
1006154
1006168
1006175
1006176
1006181
1006187
1006190
1006192
1006193
1006202
1006210
1006216
1006217
1006231
1006232
1006233
1006235
1006240
1006241
1006242
1006243
1006244
1006246
1006248
1006253
1006255
1006259
1006264
1006271
1006273
1006276
1006278
1006280
1006281
1006282
1006283
1006284
1006291
1006292
1006299
1006312
1006321
1006324
1006328
1006330
1006332
1006334
1006346
1006347
1006351
1006640
1006651
1006661
1006742
1006774
1008682
1008726
1008754
1008810
1008829
1008911
1008916
1008924
1008925
1008928
